<a href="https://colab.research.google.com/github/Mel-Z/Motors-On-Stress-Off/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
import json
import os
from tqdm.auto import tqdm
import shutil
import random
import cv2

# Instalar la librería de Kaggle para descargar datasets
!pip install kaggle

# Antes de descargar el dataset, es importante definir las credenciales para acceder al API de Kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

# Puedes crear tu propio token y username de la API de Kaggle en https://www.kaggle.com/
api_token = {"username":"benjaminvillegas","key":"51aee955395f28ce555fbc0a8abb4481"}

# Crear un archivo con las credenciales, de tal forma que kaggle pueda leerlas facilmente
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
%cd /content

# Comprobar si el conjunto de datos ya se ha descargado
if not os.path.exists('./affectnethq'):
  os.makedirs('affectnethq')
else: 
  !rm -rf affectnethq

# Descargar un dataset desde Kaggle
#!kaggle datasets download -d tunhunhminh/demodata -p city_problems
!kaggle datasets download -d tom99763/affectnethq



/content
100% 8.10G/8.12G [00:54<00:00, 193MB/s]
100% 8.12G/8.12G [00:54<00:00, 160MB/s]


In [ ]:
# Descomprimir dataset 
!unzip -qn '/content/affectnethq.zip' -d /content/affectnethq > /dev/null
!rm /content/affectnethq.zip

In [ ]:
categories = ["surprise","happy","anger","disgust","fear","contempt","sad","neutral"]
dataset = pd.read_csv('/content/affectnethq/labels.csv')

In [ ]:
dataset["label"][0]

'surprise'

In [ ]:
cad=dataset["pth"][0]
s=cad.split('/')
cad2=s[1]
cad2

'image0000006.jpg'

In [ ]:
#movemos cada imagen a su carpeta respectiva segun el archivo csv
for idx, sample in enumerate(dataset["label"]):
  s=dataset["pth"][idx]
  cad=s.split('/')
  direc=cad[1]
  shutil.move("/content/affectnethq/"+dataset["pth"][idx],"/content/affectnethq/"+dataset["label"][idx]+"/"+direc)


In [ ]:
#comprobamos que el nuero de imagenes corresponda a cierta categoria segun e
import os
initial_count = 0
dir = "/content/affectnethq/sad"
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count)

3344


In [ ]:
count=0
for i, sample in enumerate(dataset["label"]):
  if sample=='sad':
    count=count+1
print(count)

3352


In [ ]:
#dividimos el dataset
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders  

#Dividir una carpeta en 80% train y 20% test
splitfolders.ratio("/content/affectnethq", output="/content/dataAffect2", seed=1337, ratio=(.7, .15, .15), group_prefix=None) # default values

Copying files: 30624 files [02:22, 215.23 files/s]


In [ ]:
# Buscar dentro los directorios
data_dir = '/content/dataAffect2'
print(os.listdir(data_dir))
classes_train = os.listdir(data_dir + "/train")
classes_valid = os.listdir(data_dir + "/val")
classes_test = os.listdir(data_dir + "/test")
print(f'Train Classes - {classes_train}')
print(f'Validation Classes - {classes_valid}')
print(f'Test Classes - {classes_test}')

['train', 'val', 'test']
Train Classes - ['contempt', 'happy', 'disgust', 'fear', 'sad', 'neutral', 'anger', 'surprise']
Validation Classes - ['contempt', 'happy', 'disgust', 'fear', 'sad', 'neutral', 'anger', 'surprise']
Test Classes - ['contempt', 'happy', 'disgust', 'fear', 'sad', 'neutral', 'anger', 'surprise']


In [ ]:
#utilizaremos el modelo pre entrenado de efficientnet
!pip install -U efficientnet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 7.3 MB/s 


In [ ]:
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [ ]:
# Todas las imágenes seran divididas entre el maximo valor por canal, por lo que quedaran rescaladas entre el 0 y 1. 
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Tamaño del lote (imágenes a procesar por el modelo al mismo tiempo)
BATCH_SIZE = 32
TRAINSET_PATH="/content/dataAffect2/train"
VALIDSET_PATH = "/content/dataAffect2/val"
TESTSET_PATH = "/content/dataAffect2/test"


# Importar datos desde los directorios y convertirlos en lotes de tensores
train_data = train_datagen.flow_from_directory(TRAINSET_PATH,
                                               batch_size=BATCH_SIZE, # tamaño del lote
                                               target_size=(224, 224), # convertir todas las imágenes a 224 x 224
                                               class_mode="categorical", # tipo de problema, usar 'binary' si tuviesemos dos clases
                                               seed=42) # semilla para generar las mismas selecciones durante los experimentos

valid_data = valid_datagen.flow_from_directory(VALIDSET_PATH,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)
test_data = test_datagen.flow_from_directory(TESTSET_PATH,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)


Found 21433 images belonging to 8 classes.
Found 4590 images belonging to 8 classes.
Found 4601 images belonging to 8 classes.


In [ ]:
#cargamos el modelo efficientNet de keras
from tensorflow.keras.applications import EfficientNetB0
base_model = EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet',classes=8)

In [ ]:
#congelamos layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = tf.keras.layers.Dense(8, activation="sigmoid")(x)
model_final = Model(base_model.input, predictions)

In [ ]:
model_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])               

In [ ]:
eff_history = model_final.fit_generator(train_data, validation_data = valid_data, epochs = 10,steps_per_epoch = 100) #steps_per_epoch = 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
 95/100 [===========================>..] - ETA: 13s - loss: 3.0781 - accuracy: 0.1368